In [1]:
# Move To Correct Directory
import os
os.chdir("C:\\Users\Chris\Desktop\Linkbuilder stuff\TotalCrypto Data")

In [2]:
import pandas as pd

In [3]:
# Get the data
BTC_df = pd.read_csv("bitcoinity_data_BTC_price_5y.csv", index_col=0)
Commodities_df = pd.read_csv("IMF_Commodity_data_csv.csv", header = 1, index_col=0)

In [4]:
Commodities_df = Commodities_df.rename_axis('Date')
Commodities_df.head()

,"All Commodity Price Index, 2016 = 100, includes both Fuel and Non-Fuel Price Indices","Commodities for Index: All, excluding Gold, 2016 = 100","Non-Fuel Price Index, 2016 = 100, includes Precious Metal, Food and Beverages and Industrial Inputs Price Indices","Food and Beverage Price Index, 2016 = 100, includes Food and Beverage Price Indices","Food Price Index, 2016 = 100, includes Cereal, Vegetable Oils, Meat, Seafood, Sugar, and Other Food (Apple (non-citrus fruit), Bananas, Chana (legumes), Fishmeal, Groundnuts, Milk (dairy), Tomato (veg)) Price Indices","Beverage Price Index, 2016 = 100, includes Coffee, Tea, and Cocoa","Industrial Inputs Price Index, 2016 = 100, includes Agricultural Raw Materials and Base Metals Price Indices","Agriculture Price Index, 2016 = 100, includes Food and Beverages and Agriculture Raw Materials Price Indices","Agricultural Raw Materials Index, 2016 = 100, includes Timber, Cotton, Wool, Rubber, and Hides Price Indices","All Metals Index, 2016 = 100: includes Metal Price Index (Base Metals) and Precious Metals Index",...,"Palladium, LME spot price, USD/ troy ounce","Platinum, LME spot price, USD/troy ounce",North American Spot LPG Propane Price/Mont Belvieu LST,"US Gulf NOLA Urea Granular Spot Price, USD/ST","Potassium Chloride (Muriate of Potash) Standard Grade: FOB Vancouver Spot Price, USD/metric tonne","US Gulf NOLA DAP Export Spot Price per MT, USD/metric tonne","Monthly average consumer prices in metropolitan France - Tomatoes (1 Kg), EUR","USDA Class 3 (formerly known as Basic Formula) Milk Spot Price, USD/cwt","MCX India Chana Spot, INR/100 Kgs","Monthly average consumer prices in metropolitan France - Apples (1 Kg), EUR"
Date,,,,,,,,,,,,,,,,,,,,,
Data Type,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
Frequency,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,...,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly
1980M1,NaN,NaN,NaN,NaN,NaN,124.374,NaN,NaN,69.466,NaN,...,NaN,806.79,NaN,150.00,100.00,261.32,NaN,NaN,NaN,NaN
1980M2,NaN,NaN,NaN,NaN,NaN,124.357,NaN,NaN,72.604,NaN,...,NaN,882.32,NaN,160.71,115.00,258.10,NaN,NaN,NaN,NaN
1980M3,NaN,NaN,NaN,NaN,NaN,129.212,NaN,NaN,68.529,NaN,...,NaN,746.54,NaN,165.00,115.00,259.86,NaN,NaN,NaN,NaN


In [75]:
## Clean up commodities data
# Start by removing years for which bitcoin data does not cover (prior to 2014)
# This will also remove all the NaN values from earlier years
clean_comm_df = Commodities_df.loc['2014M3':'2019M2'] # Note we start in March becase that's where the BTC data starts 
clean_comm_df = clean_comm_df.convert_objects(convert_numeric=True)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """


In [76]:
clean_comm_df.dtypes

All Commodity Price Index, 2016 = 100, includes both Fuel and Non-Fuel Price Indices                                                                                                                                        float64
Commodities for Index: All, excluding Gold, 2016 = 100                                                                                                                                                                      float64
Non-Fuel Price Index, 2016 = 100, includes Precious Metal, Food and Beverages and Industrial Inputs Price Indices                                                                                                           float64
Food and Beverage Price Index, 2016 = 100, includes Food and Beverage Price Indices                                                                                                                                         float64
Food Price Index, 2016 = 100, includes Cereal, Vegetable Oils, Meat, Seafood, Sugar, and

In [77]:
## Clean up BTC data
# Convert time column to datetime and make index
BTC_df.index = pd.to_datetime(BTC_df.index)
# Label other column

In [78]:
# Calculate monthly average so it matches commodities data
BTC_monthly = BTC_df.resample('M').mean()

In [79]:
# Drop last row of BTC-monthly because it goes beyond the commodities data 
BTC_monthly.drop(BTC_monthly.index[-1], inplace=True)

In [80]:
# Check start and end values
print(clean_comm_df.index[-1])
print(BTC_monthly.index[-1])
print(clean_comm_df.index[0])
print(BTC_monthly.index[0])
print(len(BTC_monthly.index))
print(len(clean_comm_df.index))

2019M2
2019-02-28 00:00:00
2014M3
2014-03-31 00:00:00
60
60


In [81]:
# Create a df that is the averages of the rows
mean_BTC_price = BTC_monthly.mean(axis=1, numeric_only=True)
mean_BTC_price.head()

Time
2014-03-31    549.933532
2014-04-30    463.034015
2014-05-31    484.326445
2014-06-30    614.984811
2014-07-31    618.147352
Freq: M, dtype: float64

In [82]:
# Append onto commodity data
clean_comm_df['Average Bitcoin Price in USD'] = mean_BTC_price.values
month_df = clean_comm_df
# Clean up index
month_df.index = BTC_monthly.index


In [84]:
# Export data as a csv so I can transfer to Infogram because my SQL skills are not upto piping it in direct :(
month_df.round(2)
month_df.to_csv(r'C:\\Users\Chris\Desktop\Linkbuilder stuff\TotalCrypto Data\bitcoin_vs_commodity_data_5yr_rounded.csv')

month_df is now the giant dataframe that we can use to compare prices of commodities with BTC. 
Next section will look at doing some stuff with that data. 

In [167]:
# This compares BTC with Gold and Platinum
month_df_precious = month_df[['Average Bitcoin Price in USD', 'Platinum, LME spot price, USD/troy ounce', 'Gold, Fixing Committee of the London Bullion Market Association, London 3 PM fixed price, US$ per troy ounce']]
month_df_precious.columns = ['BTC' , 'Platinum', 'Gold']

In [168]:
#Let's look at crude oil price vs. BTC
Oil_cols = [col for col in month_df.columns if 'Crude Oil' in col]
BTC_col_name = 'Average Bitcoin Price in USD'
Oil_df = month_df[Oil_cols + [BTC_col_name]]

In [158]:
# How many barrels of oil could you buy at peak BTC with a single bitcoin? 
peak_btc_idx = Oil_df[BTC_col_name].idxmax()
(Oil_df.loc[peak_btc_idx][-1]/Oil_df.loc[peak_btc_idx][1]).round(0)

245.0

In [157]:
# Now, how much uraniam would 1BTC have got you? 
Uranium_cols = [col for col in month_df.columns if 'Uranium' in col]
Uranium_df = month_df[Uranium_cols + [BTC_col_name]]
(Uranium_df.loc[peak_btc_idx][-1]/Uranium_df.loc[peak_btc_idx][1]).round(0)

728.0

In [169]:
# Now let's actually make some graphs 
import numpy as np
import seaborn as sns